In [1]:
import plotly.offline as pyo
import plotly.graph_objs as go
import pandas as pd

A notebook to work on data prep and plotly visualizations so that I have things set up in a way that can be easily plugged in to the final dashboard. I'll be using plotly offline, so I'll simply insert static images screencapped from the html objects.

First a bit of data prep, using the city master csvs I prepared in the first big_deal notebook.

In [10]:
df = pd.read_csv('New_York_NY_master.csv')
df.head()

,years,TMAX,TMIN,t90,t95,t100
0,1947,46.821581,30.813200,0,0,0
1,1948,61.402767,42.641671,9,4,0
2,1949,64.488553,45.420937,20,8,0
3,1950,60.902360,43.011863,5,0,0
4,1951,62.462723,44.219303,1,0,0


A couple of quick prep steps - resetting the index, cutting off the first year (which is actually only a partial year's worth of data) and adding in trailing average columns, something I really should have done in my .py file that called the NOAA API and formatted these dataframes.

In [11]:
df.drop(0,inplace=True)
for metric in ['TMAX','TMIN','t90']:
    df[f'{metric}_rolling'] = [df[metric][0:n+1].mean() if n<10 else df[metric][n-10:n+1].mean() for
                              n in range(0,len(df))]
    
df.set_index('years',inplace=True)
df.head()

,TMAX,TMIN,t90,t95,t100,TMAX_rolling,TMIN_rolling,t90_rolling
years,,,,,,,,
1948,61.402767,42.641671,9,4,0,61.402767,42.641671,9.000000
1949,64.488553,45.420937,20,8,0,62.945660,44.031304,14.500000
1950,60.902360,43.011863,5,0,0,62.264560,43.691490,11.333333
1951,62.462723,44.219303,1,0,0,62.314101,43.823443,8.750000
1952,62.634314,44.782412,12,1,0,62.378143,44.015237,9.400000


As of my beginning this notebook, I'm still a little unsure what I want the final visualizations to look like. Part of the goal here is exploratory and simply continuing to get a better feel for the plotly layout.

Part of what makes plotly so excellent is that the default styling, without a whole lot of extra work, already looks pretty good.

In [20]:
metric = 't90'
data = [go.Scatter(x=df.index, y=df[column], mode='lines') for column in [metric,f'{metric}_rolling']]
layout = go.Layout(title = f'{metric} and 10 year rolling average')
fig = go.Figure(data = data, layout=layout)
pyo.plot(fig)

'temp-plot.html'

Produces:
![basic_no_formatting.png](https://github.com/max-miller/big_deal/blob/master/visualizations/basic_no_formatting.png?raw=true)

This does remind that at the very least I need to give the traces their appropriate names

In [18]:
metric = 'TMAX'
data = [go.Scatter(x=df.index, y=df[column], mode='lines', name=column) for column in [metric,f'{metric}_rolling']]
layout = go.Layout(title = f'{metric} and 10 year rolling average')
fig = go.Figure(data = data, layout=layout)
pyo.plot(fig)

'temp-plot.html'

![with_names.png](https://github.com/max-miller/big_deal/blob/master/visualizations/with_names.png?raw=true)

In [19]:
metric = 'TMIN'
data = [go.Scatter(x=df.index, y=df[column], mode='lines', name=column) for column in [metric,f'{metric}_rolling']]
layout = go.Layout(title = f'{metric} and 10 year rolling average')
fig = go.Figure(data = data, layout=layout)
pyo.plot(fig)

'temp-plot.html'

The most important thing for this project, interactivity wise, is the ability to select metric or city. For that I'm going to try using Dash.

I'm a little new to this. But the first simple thing is to set up Dash to take in a metric (TMAX, TMIN, t90) and return the appropriate graph. 

In [22]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [27]:
app= dash.Dash() #create Dash app
#layout for now is simple, just the graph and the dropdown with metrics.
app.layout = html.Div([
                dcc.Graph(id='graph'),
                dcc.Dropdown(id='metric-dropdown',
                            options=[{'label':'Average Maximum Temp','value':'TMAX'},
                                    {'label':'Average Minimum Temp','value':'TMIN'},
                                    {'label':'Number of days above 90','value':'t90'}])])

#callback is what actually links the input from the dropdown to some other component, here I specify
#that the output will be the component with the id 'graph'
@app.callback(Output('graph','figure'),
             [Input('metric-dropdown','value')])

#This function creates the appropriate plotly style plot from the inputted metric.
def update_figure(metric):
    data = [go.Scatter(x=df.index, y=df[column], mode='lines', name=column) 
            for column in [metric,f'{metric}_rolling']]
    layout = go.Layout(title = f'{metric} and 10 year rolling average')
    return {'data':data,'layout':layout}


app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Aug/2019 17:37:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 17:37:52] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1561062262 HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 17:37:52] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1561062262 HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 17:37:52] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1561062262 HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 17:37:52] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1561063963 HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 17:37:52] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.0.0&m=1563307603 HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 17:37:52] "GET /_dash-component-suites/dash_renderer/dash_renderer.min.js?v=1.0.0&m=1561062262 HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2019 1

Having never made interactive visualizations like this before, I'm very happy with how easy it was to create these, and it's clear to me how to extend this to encompass a bunch of other things I'll want to do.

![TMAX](https://github.com/max-miller/big_deal/blob/master/visualizations/tmax_dash.png?raw=true)

![TMIN](https://github.com/max-miller/big_deal/blob/master/visualizations/tmin_dash.png?raw=true)

One quirk I need to figure out is how to get it to default to a value, since currently, when I start it up, there is no default graph populated as it waits for the user to input a metric:

![TMIN](https://github.com/max-miller/big_deal/blob/master/visualizations/empty_graph.png?raw=true)
